In [9]:
from ase.md.langevin import Langevin
from ase.build import molecule
from ase import Atoms
from xtb.ase.calculator import XTB
from ase.calculators.emt import EMT
from ase import units
from rich import print
from ase.calculators.acn import ACN, m_me, r_cn, r_mec
from ase.constraints import FixLinearTriatomic
from ase.io import Trajectory
from ase.md import MDLogger
import numpy as np

In [10]:
from xtb.utils import get_method
get_method('GFN2-xTB') is not None

True

In [11]:
smile = 'CCCCCC'

In [12]:
# Bond lengths
from ase.data.pubchem import pubchem_atoms_search, pubchem_atoms_conformer_search
# Create butene molecule
# atoms = Atoms('CCCCHHHHHHHH', positions=positions)
# atoms.rotate(30, 'x')
atoms = pubchem_atoms_search(smiles=smile)
# Now use this butene object in your MD simulation

/home/amir/.conda/envs/venv/lib/python3.10/site-packages/ase/data/pubchem.py:80: UserWarning: The structure "CCCCCC" has more than one conformer in PubChem. By default, the first conformer is returned, please ensure you are using the structure you intend to or use the `ase.data.pubchem.pubchem_conformer_search` function
  warnings.warn(


In [13]:
print(atoms)

Atoms(symbols='C6H14', pbc=False, initial_charges=...)

In [14]:
d = 0.776 / 1e24
L = ((atoms.get_masses().sum() / units.mol) / d)**(1 / 3.) * 10
# Set up box of 27 acetonitrile molecules
atoms.set_cell((L, L, L))
atoms.center()
# atoms = atoms.repeat((3, 3, 3))
print(atoms)

Atoms(symbols='C6H14', pbc=False, cell=[56.91952406730776, 56.91952406730776, 56.91952406730776], 
initial_charges=...)

In [15]:
atoms.calc = XTB(
    method="GFN2-xTB",
    directory='xtb_files',
    atoms=atoms,
    charge=0,
    multiplicity=1,
    parallel=True,
    periodic=True,
)
print(atoms.get_potential_energy())
print(atoms.get_forces())

-544.0476194232137

[[-0.01528865 -0.11282544  0.01029797]
 [ 0.01530302  0.11638278 -0.02871846]
 [ 0.02246106 -0.00968628  0.02209957]
 [-0.02437843  0.00671538  0.00050757]
 [ 0.34257072  0.07428151 -0.00791582]
 [-0.33670484 -0.07825666  0.00894929]
 [-0.01036006 -0.12280372  0.09080527]
 [ 0.00373988 -0.12172686 -0.08765412]
 [ 0.00377473  0.12220927 -0.09418383]
 [ 0.00144297  0.12133037  0.08193103]
 [-0.05272219  0.09898128 -0.07643699]
 [-0.04882524  0.09866721  0.07858461]
 [ 0.04700164 -0.09889124 -0.07808736]
 [ 0.05496966 -0.10005602  0.080397  ]
 [-0.0425383  -0.12032151  0.17873964]
 [-0.1305113   0.14830843  0.00149185]
 [-0.04477943 -0.11773873 -0.18362444]
 [ 0.043107    0.12287668  0.18311784]
 [ 0.04435971  0.1184588  -0.18320517]
 [ 0.12737808 -0.14590526  0.00290455]]

In [17]:
atoms.calc

In [22]:
dyn = Langevin(
    atoms,
    timestep=0.5 * units.fs,
    temperature_K=298,
    friction=0.02,
    trajectory=f'{smile}.traj',
    logfile=f'{smile}.log',
    append_trajectory=True)


# Set up trajectory
traj = Trajectory(f'{smile}.traj', 'w', atoms)
dyn.attach(traj.write, interval=100)
# dyn.attach(MDLogger(
#     dyn,
#     atoms,
#     'md.log',
#     header=False,
#     stress=False,
#     peratom=True,
#     mode="a"
#     ), interval=1000)

In [20]:
from logmd import LogMD
logmd = LogMD(project="Alkanes")

initial_pos = atoms.positions.copy()
def log(atoms, dyn):
    # disulfur-bridge distance
    # sulfur_indices = [i for i, atom in enumerate(atoms) if atom.symbol == 'S']
    # sulfurs = atoms[sulfur_indices]
    # S_bridge = np.linalg.norm(sulfurs[1].position - sulfurs[4].position)

    # atom change
    # atom_change = np.mean(np?.linalg.norm(initial_pos-atoms.positions, axis=1))

    data_dict = {
        # "S_bridge": f"{S_bridge} [A]",
        # "any_name_goes!<3": f"{atom_change} [A]",
    }

    logmd(atoms, dyn, data_dict=data_dict )

dyn.attach(lambda: log(atoms, dyn), interval=100)

FileNotFoundError: [Errno 2] No such file or directory: ''

In [23]:
dyn.run(500_000)

True

In [ ]:
# Create a temperature monitoring function with a moving window
def run_until_temp_converges(dynamics,
                             step: int = 1,
                             target_temp: float = 298.0,
                             window_size: int = 10_000,
                             max_steps: int = 1_000_000,
                             tol: float = 1.0):
    """
    Run dynamics until the average temperature over the last window_size steps is
    within tol of target_temp, or until max_steps is reached.

    Parameters:
    -----------
    dynamics : ASE dynamics object
        The dynamics to run
    step : int
        Run dynamics for this many steps at a time
    target_temp : float
        Target temperature in Kelvin
    window_size : int
        Number of steps to average over
    max_steps : int
        Maximum number of steps to run
    tol : float
        Tolerance for temperature convergence

    Returns:
    --------
    int
        Number of steps actually run
    """
    # History to store temperatures
    temp_history = []

    # Steps counter
    steps_run = 0

    # Main loop
    while steps_run < max_steps:
        # Run one step
        dynamics.run(step)
        steps_run += step

        # Get current temperature and add to history
        current_temp = dynamics.atoms.get_temperature()
        temp_history.append(current_temp)

        # Keep only the last window_size temperatures
        if len(temp_history) > window_size:
            temp_history.pop(0)

        # Check if we have enough data and if temperature has converged
        if len(temp_history) == window_size:
            avg_temp = sum(temp_history) / window_size
            if abs(avg_temp - target_temp) <= tol:
                print(f"Temperature converged to {avg_temp:.2f}K after {steps_run} steps")
                return steps_run

        # Print progress every 1000 steps
        if steps_run % 1000 == 0:
            if len(temp_history) == window_size:
                avg_temp = sum(temp_history) / window_size
                print(f"Step {steps_run}: Average temperature over last {window_size} steps: {avg_temp:.2f}K")
            else:
                print(f"Step {steps_run}: Collecting temperature data ({len(temp_history)}/{window_size})")

    print(f"Maximum steps ({max_steps}) reached without convergence")
    return steps_run

# Replace your dyn.run(500_000) with:
run_until_temp_converges(dyn, target_temp=298.0, window_size=10000, max_steps=500000, tol=1.0)

In [26]:
traj = Trajectory(f'{smile}_after.traj', 'w', atoms)
dyn.attach(traj.write, interval=5)
new_logger = MDLogger(
    dyn,
    atoms,
    f'{smile}_after.log',
    header=True,
    stress=False,
    peratom=True,
    mode="a"
)
dyn.attach(new_logger, interval=5)
dyn.run(2000)

True

In [ ]:
print(atoms.get_potential_energy())
print(atoms.get_forces())

In [ ]:
print(atoms.get_potential_energy())
print(atoms.get_forces())

In [ ]:
import plotly.express as px
import pandas as pd

# Load the data (replace 'your_file.txt' with the actual filename)
df = pd.read_csv("md.log", delim_whitespace=True)

# Plot Time vs. Etot
fig = px.line(df, x="Time[ps]", y="Etot[eV]", title="Total Energy vs. Time")

# Show the interactive plot
fig.show()

In [ ]:
import plotly.express as px
import pandas as pd

# Load the data (replace 'your_file.txt' with the actual filename)
df = pd.read_csv("md.log", delim_whitespace=True)

# Plot Time vs. Etot
fig = px.line(df, x="Time[ps]", y="T[K]", title="T[K] vs. Time")

# Show the interactive plot
fig.show()

In [ ]:
!ls

In [ ]:
from ase.io import read
from ase.visualize import view

# Read the entire trajectory
traj = read("md.traj", index=":")  # ":" means all frames
view(traj)  # Opens a simple GUI viewer

In [ ]:
# Pull out specific snapshots (e.g., the last frame):
last_frame = read("mymd.traj", index=-1)  # Last frame
print(last_frame.get_positions())  # Atomic coordinates

In [ ]:
from ase.io import write
write("mymd.xyz", traj)  # Creates a multi-frame XYZ file

In [ ]:
for frame in traj:
    energy = frame.get_potential_energy()
    print(f"Energy: {energy} eV")

In [ ]:
import numpy as np
from ase.io import read, write
from ase.db import connect

# Parameters for averaging
start_frame = 400000
end_frame = 500000
window_size = 100

# Load frames from your trajectory
traj = read("md.traj", index=":")

# Check trajectory length to avoid indexing errors
total_frames = len(traj)
print(f"Total frames available: {total_frames}")

# Adjust end_frame if exceeds total_frames
end_frame = min(end_frame, total_frames)

# Establish connection to the ASE database for averaged frames
db_average = connect('averaged_simulation_data.db')

# Prepare a list for averaged frames to save as xyz
averaged_frames_xyz = []

# Loop through trajectory frames starting from start_frame, apply moving average every window_size
for i in range(start_frame, end_frame, window_size):
    window_frames = traj[i:i + window_size]

    # Check window completeness
    if len(window_frames) < window_size:
        print(f"Less than {window_size} frames remain; skipping remaining {len(window_frames)} frames")
        break

    # Compute average positions, forces, and energies
    avg_positions = np.mean([frame.get_positions() for frame in window_frames], axis=0)
    avg_forces = np.mean([frame.get_forces() for frame in window_frames], axis=0)
    avg_energy = np.mean([frame.get_potential_energy() for frame in window_frames])

    # Use the atomic information from the first frame in the window to create a new ASE Atoms object
    avg_atoms = window_frames[0].copy()
    avg_atoms.set_positions(avg_positions)

    # Store average information to the ASE Database
    db_average.write(avg_atoms, data={
        'average_energy': avg_energy,
        'average_forces': avg_forces
    })

    # Also collect averaged atoms for XYZ file export
    averaged_frames_xyz.append(avg_atoms)

# Write averaged frames into XYZ file
write("averaged_frames.xyz", averaged_frames_xyz)
